# 🛡️ LOGISTIC REGRESSION WITH REGULARIZATION

## WHAT IS REGULARIZATION?
- IN REAL-WORLD MACHINE LEARNING, MODELS DO NOT FAIL BECAUSE THEY ARE TOO SIMPLE, THEY FAIL BECAUSE THEY **LEARN TOO MUCH**.
- THIS PHENOMENON IS CALLED **OVERFITTING**.
- REGULARIZATION IS A TECHNIQUE USED TO PREVENT **OVERFITTING** BY PENALIZING LARGE WEIGHTS ($\theta$).
- IT ADDS A `"COMPLEXITY TERM"` TO THE COST FUNCTION, FORCING THE MODEL TO KEEP PARAMETERS SMALL AND SIMPLE.

> OVERFITTING OCCURS WHEN A MODEL:
- MEMORIZES TRAINING DATA
- LEARNS NOISE INSTEAD OF PATTERNS
- PERFORMS WELL ON TRAIN DATA BUT FAILS ON UNSEEN DATA

> REGULARIZATION IS A **CONTROL MECHANISM** THAT:
- PENALIZES LARGE WEIGHTS
- LIMITS MODEL COMPLEXITY
- IMPROVES GENERALIZATION
- INTRODUCES BIAS TO REDUCE VARIANCE

LOGISTIC REGRESSION LEARNS PARAMETERS BY MINIMIZING A COST FUNCTION.

WITHOUT REGULARIZATION:

## REGULARIZATION IN LOGISTIC REGRESSION

COST FUNCTION WITHOUT REGULARIZATION 

    J(θ) = -(1/m) Σ [y·log(hθ(x)) + (1 − y)·log(1 − hθ(x))]

### THE THREE TYPES WE WILL IMPLEMENT:

> **1. L2 REGULARIZATION (RIDGE)**
> * **PENALTY:** SQUARED MAGNITUDE OF WEIGHTS ($\theta_j^2$).
> * **EFFECT:** SHRINKS ALL COEFFICIENTS TOWARDS ZERO EVENLY. GOOD FOR COLLINEARITY. SMOOTH DECISION BOUNDARY
> * **MATH:**
>   $$J(\theta)_{reg} = J(\theta) + \frac{\lambda}{2m} \sum_{j=1}^{n} \theta_j^2$$

> **2. L1 REGULARIZATION (LASSO)**
> * **PENALTY:** ABSOLUTE VALUE OF WEIGHTS ($|\theta_j|$).
> * **EFFECT:** CAN SHRINK COEFFICIENTS TO EXACTLY ZERO. PERFORMS **FEATURE SELECTION**. CREATES SPARSE MODELS.
> * **MATH:**
>   $$J(\theta)_{reg} = J(\theta) + \frac{\lambda}{m} \sum_{j=1}^{n} |\theta_j|$$

> **3. ELASTIC NET**
> * **PENALTY:** A COMBINATION OF BOTH L1 AND L2.
> * **EFFECT:** BALANCES SPARSITY (L1) AND STABILITY (L2).
> * **MATH:**
>   $$J(\theta)_{reg} = J(\theta) + r \cdot (\text{L1}) + (1-r) \cdot (\text{L2})$$

---
`NOTE`: WE DO NOT REGULARIZE THE BIAS TERM ($\theta_0$). REGULARIZATION IS APPLIED **ONLY TO WEIGHTS.**